In [13]:
# import main libraries
import json
import requests
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [14]:
# import all datasets
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/incidentes2019.json")
incidents_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json")
distances_stations_ntas_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones.json")
distances_stations_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/barrios.json")
ntas_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/estaciones.json")
stations_db = json.loads(response.text)

In [15]:
len(ntas_db)

195

In [16]:
# add pupulation density to each neighborhood
for neighbor in ntas_db:
    population = neighbor["population"]
    
    shape_area = neighbor["shape_area"] / 10000000 # the shape area value was too big
    
    neighbor["population_density"] = population / shape_area

In [17]:
# iterating all neighborhoods
for neighbor in ntas_db:
    # define vehicle types
    vehicle_types = ["engine", "ladder", "rescue", "squad", "hazardous"]
    # get scal
    nta = neighbor["nta"]
    # get population
    population = neighbor["population"]
    
    # iterate vehicle types
    for vehicle_type in vehicle_types:
        # set index name
        vehicles_needed_first_shift = vehicle_type + "_needed_first_shift"
        vehicles_needed_second_shift = vehicle_type + "_needed_second_shift"

        # define counters for vehicle type needed per person, for each shift
        neighbor[vehicles_needed_first_shift] = 0
        neighbor[vehicles_needed_second_shift] = 0

        # set index name
        vehicles_needed_per_person_first_shift = vehicle_type + "_needed_per_person_first_shift"
        vehicles_needed_per_person_second_shift = vehicle_type + "_needed_per_person_second_shift"

        # define counters for vehicle type needed per person, for each shift
        neighbor[vehicles_needed_per_person_first_shift] = 0
        neighbor[vehicles_needed_per_person_second_shift] = 0

        # set index name
        #incident_average_duration_first_shift = vehicle_type + "_incident_average_duration_first_shift"
        #incident_average_duration_second_shift = vehicle_type + "_incident_average_duration_second_shift"

        # define variables for the vehicle incident average duration, for each shift
        #neighbor[incident_average_duration_first_shift] = 0.
        #neighbor[incident_average_duration_second_shift] = 0.

        # iterate all the incidents
        for incident in incidents_db:
            # get incident nta
            incident_nta = incident["nta"]
            # check if the incident happened in the actual neighborhood
            if incident_nta == nta:
                # get the vehicles needed for the incident
                vehicles_needed_for_incident = incident["units"].count(vehicle_type)
                # get incident duration
                incident_duration = incident["incident_duration"]
                # get the shift
                is_first_shift = incident["is_first_shift"]
                
                # check if the vehicle was used in that incident
                if vehicles_needed_for_incident != 0:
                    # check the shift
                    if is_first_shift == True:

                        # sum how many vehicles of that type was used in this incident
                        neighbor[vehicles_needed_per_person_first_shift] += vehicles_needed_for_incident

                        # check if we have the data about incident duration
                        #if incident_duration != None:
                            # multiply the incident duration for vehicles
                            #incident_duration *= vehicles_needed_for_incident
                            # sum incidents durations
                            #neighbor[incident_average_duration_first_shift] += incident_duration

                    # for the second shift, do the same operations of the first shift
                    else:
                        neighbor[vehicles_needed_per_person_second_shift] += vehicles_needed_for_incident

                        #if incident_duration != None:
                            #incident_duration *= vehicles_needed_for_incident
                            #neighbor[incident_average_duration_second_shift] += incident_duration

        # copy the number of vehicles needed per neighborhood
        neighbor[vehicles_needed_first_shift] = neighbor[vehicles_needed_per_person_first_shift]
        neighbor[vehicles_needed_second_shift] = neighbor[vehicles_needed_per_person_second_shift]

        # check if the vehicles needed for the first shift are not 0
        #if neighbor[vehicles_needed_per_person_first_shift] != 0:
            # get average duration of an incident on the first shift
            #neighbor[incident_average_duration_first_shift] /= neighbor[vehicles_needed_per_person_first_shift]
        
        # check if the vehicles needed for the second shift are not 0
        #if neighbor[vehicles_needed_per_person_second_shift] != 0:
            # get average duration of an incident on the second shift
            #neighbor[incident_average_duration_second_shift] /= neighbor[vehicles_needed_per_person_second_shift]

        # check if the population of the neighborhood is not 0
        if population != 0:
            # get the number of vehicles needed per person on both shifts
            neighbor[vehicles_needed_per_person_first_shift] /= population
            neighbor[vehicles_needed_per_person_second_shift] /= population

In [18]:
# iterate all neighbors
for neighbor in ntas_db:
    # define vehicle types
    vehicle_types = ["engine", "ladder", "rescue", "squad", "hazardous"]
    
    # get population density of the neighborhood
    population_density = neighbor["population_density"]
    
    # iterate all vehicle types
    for vehicle_type in vehicle_types:
        # define index names
        vehicles_needed_per_density_first_shift = vehicle_type + "_needed_per_density_first_shift"
        vehicles_needed_per_person_first_shift = vehicle_type + "_needed_per_person_first_shift"
        vehicles_needed_per_density_second_shift = vehicle_type + "_needed_per_density_second_shift"
        vehicles_needed_per_person_second_shift = vehicle_type + "_needed_per_person_second_shift"

        # get vehicles needed per neigborhood (per density) for both shifts
        neighbor[vehicles_needed_per_density_first_shift] = neighbor[vehicles_needed_per_person_first_shift] * population_density
        neighbor[vehicles_needed_per_density_second_shift] = neighbor[vehicles_needed_per_person_second_shift] * population_density

In [19]:
# get dataframe of the neighborhoods
df_ntas = pd.DataFrame(ntas_db).set_index("nta")
df_ntas

,boro_name,name,shape_area,population,population_density,engine_needed_first_shift,engine_needed_second_shift,engine_needed_per_person_first_shift,engine_needed_per_person_second_shift,ladder_needed_first_shift,...,engine_needed_per_density_first_shift,engine_needed_per_density_second_shift,ladder_needed_per_density_first_shift,ladder_needed_per_density_second_shift,rescue_needed_per_density_first_shift,rescue_needed_per_density_second_shift,squad_needed_per_density_first_shift,squad_needed_per_density_second_shift,hazardous_needed_per_density_first_shift,hazardous_needed_per_density_second_shift
nta,,,,,,,,,,,,,,,,,,,,,
QN08,Queens,St. Albans,7.741275e+07,48593,6277.131526,988,824,0.020332,0.016957,1141,...,127.627558,106.442417,147.391745,109.284326,11.496814,2.583554,5.167108,1.679310,5.296285,6.846418
BX28,Bronx,Van Cortlandt Village,2.566612e+07,50100,19519.892773,0,0,0.000000,0.000000,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
QN55,Queens,South Ozone Park,8.246139e+07,75878,9201.639284,1646,1165,0.021693,0.015354,1904,...,199.608559,141.278233,230.895928,140.550620,19.888095,6.548519,11.035467,1.455226,9.944047,7.033594
BK50,Brooklyn,Canarsie,8.208968e+07,83693,10195.313392,0,0,0.000000,0.000000,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
BX41,Bronx,Mount Hope,1.471671e+07,51807,35202.839082,0,0,0.000000,0.000000,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BK93,Brooklyn,Starrett City,1.159895e+07,13354,11513.108525,0,0,0.000000,0.000000,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
BK73,Brooklyn,North Side-South Side,2.885436e+07,45774,15863.809454,0,0,0.000000,0.000000,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
MN28,Manhattan,Lower East Side,2.330069e+07,72957,31311.095459,0,0,0.000000,0.000000,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [20]:
# interate vehicles types
for vehicle_type in vehicle_types:
    # define index names
    vehicles_needed_per_person_first_shift = vehicle_type + "_needed_per_person_first_shift"
    vehicles_needed_per_person_second_shift = vehicle_type + "_needed_per_person_second_shift"

    # drop useless columns (vehicles needed per person)
    df_ntas = df_ntas.drop(vehicles_needed_per_person_first_shift, axis=1)
    df_ntas = df_ntas.drop(vehicles_needed_per_person_second_shift, axis=1)

In [21]:
df_ntas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 195 entries, QN08 to BK99
Data columns (total 25 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   boro_name                                  195 non-null    object 
 1   name                                       195 non-null    object 
 2   shape_area                                 195 non-null    float64
 3   population                                 195 non-null    int64  
 4   population_density                         195 non-null    float64
 5   engine_needed_first_shift                  195 non-null    int64  
 6   engine_needed_second_shift                 195 non-null    int64  
 7   ladder_needed_first_shift                  195 non-null    int64  
 8   ladder_needed_second_shift                 195 non-null    int64  
 9   rescue_needed_first_shift                  195 non-null    int64  
 10  rescue_needed_second_shift 

In [22]:
# define Min-Max Scaler
MM_scaler = MinMaxScaler()

# scale the data
df_ntas.loc[:, "shape_area":"hazardous_needed_per_density_second_shift"] = MM_scaler.fit_transform(df_ntas.loc[:, "shape_area":"hazardous_needed_per_density_second_shift"])
df_ntas

,boro_name,name,shape_area,population,population_density,engine_needed_first_shift,engine_needed_second_shift,ladder_needed_first_shift,ladder_needed_second_shift,rescue_needed_first_shift,...,engine_needed_per_density_first_shift,engine_needed_per_density_second_shift,ladder_needed_per_density_first_shift,ladder_needed_per_density_second_shift,rescue_needed_per_density_first_shift,rescue_needed_per_density_second_shift,squad_needed_per_density_first_shift,squad_needed_per_density_second_shift,hazardous_needed_per_density_first_shift,hazardous_needed_per_density_second_shift
nta,,,,,,,,,,,,,,,,,,,,,
QN08,Queens,St. Albans,0.223361,0.367078,0.109918,0.292654,0.339234,0.278632,0.342927,0.260234,...,0.108119,0.088128,0.106058,0.090719,0.100143,0.082782,0.090793,0.088288,0.097795,0.105349
BX28,Bronx,Van Cortlandt Village,0.062467,0.378462,0.341809,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
QN55,Queens,South Ozone Park,0.239059,0.573192,0.161128,0.487559,0.479621,0.464957,0.469801,0.479532,...,0.169097,0.116969,0.166145,0.116673,0.173235,0.209827,0.193908,0.076507,0.183615,0.108229
BK50,Brooklyn,Canarsie,0.237903,0.632227,0.178528,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
BX41,Bronx,Mount Hope,0.028422,0.391357,0.616431,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BK93,Brooklyn,Starrett City,0.018728,0.100878,0.201604,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
BK73,Brooklyn,North Side-South Side,0.072380,0.345783,0.277788,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
MN28,Manhattan,Lower East Side,0.055112,0.551126,0.548283,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [23]:
# get the dataframe of the distances between stations and neighborhoods
df_distances_station_nta = pd.DataFrame(distances_stations_ntas_db)
df_distances_station_nta

,42 South Street,49 Beekman Street,100 Duane Street,14 N. Moore Street,75 Canal Street,25 Pitt Street,222 East 2 Street,340 East 14 Street,253 Lafayette Street,42 Great Jones Street,...,256 Hylan Blvd.,278 McClean Avenue,1850 Clove Road,1592 Richmond Road,256 Nelson Ave.,345 Annadale Road,1560 Drumgoole Road West,7219 Amboy Road,1100 Rossville Ave,305 Front Street
QN08,2123.18,2130.20,2104.71,2073.11,1896.78,1881.48,1925.06,1876.57,2030.47,1974.10,...,2449.03,2393.86,2396.74,2617.64,3444.35,3471.11,3761.95,3933.27,3559.47,2631.33
BX28,1894.39,1874.63,1935.64,2020.57,1809.95,1751.58,1728.54,1601.29,1843.17,1762.19,...,3168.88,3133.14,3051.15,3274.89,3596.90,3272.39,3250.70,3276.83,3037.71,3351.18
QN55,1872.67,1776.72,1811.35,1821.87,1646.27,1630.97,1674.55,1626.81,1779.96,1724.34,...,2113.36,2058.18,2061.06,2281.96,3108.67,3135.43,3426.27,3597.59,3223.79,2295.65
BK50,1532.61,1587.44,1614.04,1630.87,1548.42,1533.11,1576.70,1662.97,1682.11,1704.08,...,1726.71,1671.54,1674.41,1895.32,2722.02,2748.79,3039.63,3210.95,2837.15,1909.01
BX41,1774.19,1754.43,1815.43,1900.37,1689.74,1631.38,1608.33,1481.08,1722.97,1641.99,...,3048.68,3012.94,2893.91,3117.65,3439.67,3115.16,3093.46,3119.59,2880.48,3201.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BK93,1571.05,1625.88,1652.48,1669.31,1586.86,1571.56,1615.15,1701.41,1720.55,1742.52,...,1753.73,1705.62,1706.59,1922.34,2756.10,2775.81,3066.65,3243.13,2869.33,1936.03
BK73,723.32,730.33,704.84,673.24,496.91,481.61,525.20,611.46,630.60,652.57,...,1791.46,1750.57,1744.32,1960.06,2799.28,2813.53,2988.50,3097.89,2775.52,1973.75
MN28,446.34,463.45,452.35,432.20,240.24,101.92,243.76,387.74,406.88,428.85,...,1774.52,1738.78,1727.38,1943.12,2778.55,2619.51,2747.46,2856.85,2534.48,1956.81
BK40,657.93,767.28,739.37,756.19,906.25,1014.20,1085.85,1121.00,902.06,975.73,...,1058.62,1017.73,1011.48,1227.22,2066.44,2080.69,2371.53,2548.01,2174.21,1240.91


In [24]:
# export files
df_ntas.to_csv("dataset/neighborhoods_info_standardized.csv")
df_distances_station_nta.to_csv("dataset/distance_stations_neighborhoods.csv")